Notebook to collect news data. Requires to add the API key here

In [128]:
import requests

url = "https://contextualwebsearch-websearch-v1.p.rapidapi.com/api/Search/NewsSearchAPI"

querystring = {"fromPublishedDate":"01-21-2019","toPublishedDate":"01-22-2019","autoCorrect":"false","pageNumber":"1","pageSize":"50","q":"Microsoft Corp","safeSearch":"false"}

headers = {
    'x-rapidapi-host': "contextualwebsearch-websearch-v1.p.rapidapi.com",
    'x-rapidapi-key': "*ENTER KEY HERE*"
    }

response = requests.request("GET", url, headers=headers, params=querystring)

print(response)

<Response [200]>


In [6]:
import json

In [23]:
res = json.loads(response.text)
values = res.get('value')
for i in values:
    if "en" != i.get('language'):
        continue
    url = i.get('url')
    content = i.get('description')
    body = i.get('body')
    print((content))
    if len(content) == 0:
        content = body
    elif len(body) != 0:
        content += body
    outlet = (i.get('provider')).get('name')
    published_date = i.get('datePublished')
    print('url = {},content = {}, outlet = {}, published_date = {} '.format(url,content,outlet,published_date))

Updated analyst estimates for <b>Microsoft Corp</b>. - including MSF earnings per share estimates and analyst recommendations.
url = https://www.marketwatch.com/investing/stock/msf/analystestimates?countrycode=xe,content = Updated analyst estimates for <b>Microsoft Corp</b>. - including MSF earnings per share estimates and analyst recommendations.terms of use
. Historical and current end-of-day data provided by SIX Financial Information. All quotes are in local exchange time. Real-time last sale data for U.S. stock quotes reflect trades reported through Nasdaq only. Intraday data delayed at least 15 minutes or per exchange requirements.
MarketWatch Top Stories, outlet = marketwatch, published_date = 2019-01-21T00:00:00 
<b>Microsoft</b> has released an update for <b>Microsoft</b> Access 2016 64-Bit Edition. This update provides the latest fixes to <b>Microsoft</b> Access 2016 64-Bit Edition. Additionally, this update contains stability and performance improvements.
url = https://www.mi

In [2]:
import requests
url = "https://contextualwebsearch-websearch-v1.p.rapidapi.com/api/Search/NewsSearchAPI"

In [113]:
import sqlite3
create_sql = "CREATE TABLE news_table(id INTEGER PRIMARY KEY AUTOINCREMENT, published_date TEXT, url TEXT,outlet TEXT ,content TEXT);"
conn = sqlite3.connect("news.db")
conn.execute('DROP TABLE IF EXISTS news_table')
conn.execute(create_sql)
conn.close()

In [152]:
def get_data(api_url,headers, querystring, no_of_days, offset = 1):
    import datetime
    import re
    import time
    published_till = datetime.datetime(2017,4,18)
    conn = sqlite3.connect("news.db")
    cur = conn.cursor()
    try:
        while no_of_days > 0:
            published_from = published_till + datetime.timedelta(days=-offset)
            from_date = (str(published_from).split(" "))[0]
            yyyy,mm,dd = from_date.split("-")
            from_date = str(mm)+"-"+str(dd)+"-"+str(yyyy)
            to_date = (str(published_till).split(" "))[0]
            yyyy,mm,dd = to_date.split("-")
            to_date = str(mm)+"-"+str(dd)+"-"+str(yyyy)
            querystring['fromPublishedDate'] = from_date
            querystring['toPublishedDate'] = to_date
            print("from {} to {}".format(from_date, to_date))
            published_till = published_from
            no_of_days -= offset


            response = requests.request("GET", api_url, headers=headers, params=querystring)
            if response.status_code != 200:
                print("error on day {} with status code {}".format(from_date,response.status_code))
                continue
            res = json.loads(response.text)
            values = res.get('value')
            insert_list = []
            for i in values:
                #print(i)
                if "en" != i.get('language'):
                    continue
                url = i.get('url')
                outlet = (i.get('provider')).get('name')
                published_date = i.get('datePublished')
                content = i.get('description')
                body = i.get('body')

                if len(content) == 0:
                    content = body
                elif len(body) != 0:
                    content += body
                if len(published_date) == 0:
                    published_date = "UNKNOWN"
                if len(url) == 0:
                    url = "UNKNOWN"
                if len(outlet) == 0:
                    outlet = "UNKNOWN"
                #print(published_date)
                content = content.replace("'", "")
                content = content.replace('"', "")
                #content = content.replace("(", "")
                #content = content.replace(')', "")
                #content = content.replace(",",".")
                #insert_str = "('"+published_date+"','"+url+"','"+outlet+"','"+content+"');" 
                #print("inserting: {}".format(insert_str))
                #insert_list.append(insert_str)
                statement = "insert into news_table (published_date,url,outlet,content) VALUES ('"+published_date+"','"+url+"','"+outlet+"','"+content+"');"
                #print(statement)
                try:
                    cur.execute(statement)
                    conn.commit()
                except:
                    print(statement)
            #print(len(insert_list))
            #cur.executemany("insert into news_table (published_date,url,outlet,content) VALUES (?,?,?,?)",insert_list)
            #conn.commit()
            time.sleep(.300)
    finally:
        conn.close()

In [154]:
url = "https://contextualwebsearch-websearch-v1.p.rapidapi.com/api/Search/NewsSearchAPI"
headers = {
            'x-rapidapi-host': "contextualwebsearch-websearch-v1.p.rapidapi.com",
            'x-rapidapi-key': "*ENTER KEY HERE*"
            }
querystring = {
           "autoCorrect":"false",
           "pageNumber":"1",
           "pageSize":"50",
           "q":"Microsoft Corp",
           "safeSearch":"false"
            }
get_data(url, headers,querystring,250,1)

from 12-23-2017 to 12-24-2017
from 12-22-2017 to 12-23-2017
from 12-21-2017 to 12-22-2017
from 12-20-2017 to 12-21-2017
from 12-19-2017 to 12-20-2017
from 12-18-2017 to 12-19-2017
from 12-17-2017 to 12-18-2017
from 12-16-2017 to 12-17-2017
from 12-15-2017 to 12-16-2017
from 12-14-2017 to 12-15-2017
from 12-13-2017 to 12-14-2017
from 12-12-2017 to 12-13-2017
from 12-11-2017 to 12-12-2017
from 12-10-2017 to 12-11-2017
from 12-09-2017 to 12-10-2017
from 12-08-2017 to 12-09-2017
from 12-07-2017 to 12-08-2017
from 12-06-2017 to 12-07-2017
from 12-05-2017 to 12-06-2017
from 12-04-2017 to 12-05-2017
from 12-03-2017 to 12-04-2017
from 12-02-2017 to 12-03-2017
from 12-01-2017 to 12-02-2017
from 11-30-2017 to 12-01-2017
from 11-29-2017 to 11-30-2017
from 11-28-2017 to 11-29-2017
from 11-27-2017 to 11-28-2017
from 11-26-2017 to 11-27-2017
from 11-25-2017 to 11-26-2017
from 11-24-2017 to 11-25-2017
from 11-23-2017 to 11-24-2017
from 11-22-2017 to 11-23-2017
from 11-21-2017 to 11-22-2017
from 11-20

from 05-12-2017 to 05-13-2017
from 05-11-2017 to 05-12-2017
from 05-10-2017 to 05-11-2017
from 05-09-2017 to 05-10-2017
from 05-08-2017 to 05-09-2017
from 05-07-2017 to 05-08-2017
from 05-06-2017 to 05-07-2017
from 05-05-2017 to 05-06-2017
from 05-04-2017 to 05-05-2017
from 05-03-2017 to 05-04-2017
from 05-02-2017 to 05-03-2017
from 05-01-2017 to 05-02-2017
from 04-30-2017 to 05-01-2017
from 04-29-2017 to 04-30-2017
from 04-28-2017 to 04-29-2017
from 04-27-2017 to 04-28-2017
from 04-26-2017 to 04-27-2017
from 04-25-2017 to 04-26-2017
from 04-24-2017 to 04-25-2017
from 04-23-2017 to 04-24-2017
from 04-22-2017 to 04-23-2017
from 04-21-2017 to 04-22-2017
from 04-20-2017 to 04-21-2017
from 04-19-2017 to 04-20-2017
from 04-18-2017 to 04-19-2017


In [155]:
conn = sqlite3.connect("news.db")
c = conn.cursor()
for row in c.execute('SELECT count(*) FROM news_table'):
    print(row)
    
conn.close()

(23081,)
